![](https://github.com/eugenewickett/logistigate/blob/main/docs/logistigate_logo.png)

# logistigate
## Overview
Generally speaking, the logistigate methods infer aberration likelihoods at entities within a two-echelon supply chain, only using testing data from sample points taken from entities of the lower echelon. It is assumed that products originate within the system at one entity of the upper echelon, and are procured by one entity of the lower echelon. The likelihood of a lower-echelon entity obtaining product from each of the upper-echelon entities is stored in what is deemed the "transition matrix" for that system. Testing of products at the lower echelon yields aberrational (recorded as "1") or acceptable ("0") results. In what we deem the "Tracked" case, both the upper-echelon and lower-echelon entities traversed by the tested product are known upon testing. In the "Untracked" case, only the lower-echelon entity is entirely known, in addition to the system's transition matrix. It is further assumed that products are aberrational at their origin in the upper echelon with some fixed probability, and that products acceptable at the upper echelon become aberrational at the destination in the lower echelon with some other fixed probabiltiy. It is these fixed probabilities that the logistigate methods attempt to infer.

More specifically, the logistigate methods were developed with the intent of inferring sources of substandard or falsified products within a pharmaceutical supply chain. Entities of the upper echelon are referred to as "importers," and entities of the lower echelon are referred to as "outlets." This terminology is used interchangeably throughout the logistigate package.

## Installation
Before using logistigate...

In [1]:
# Make NumPy module available for numerical computations, as well as scipy and scipy.linalg
# for linear algebra computations
import numpy as np
import scipy
import scipy.linalg

### Initialize options for Newton's algorithm

In [5]:
def minimizeObjInitOptions():
    '''
    Option initialization for minimize_obj function

    Initialize algorithm options with default values

    Return values:
        options:
            This is a dictionary with fields that correspond to algorithmic
            options of our method.  In particular:

            max_iter:
                Maximum number of iterations
            tol:
                Convergence tolerance
            step_type:
                Different ways to calculate the search direction:
               'gradient_descent'
               'Newton'
               'mod_Newton
            init_alpha:
                first trial step size to attempt during line search
            suff_decrease_c1:
                coefficient in sufficient decrease condition
            output_level:
                Amount of output printed
                0: No output
                1: Only summary information
                2: One line per iteration (good for debugging)
            rho_backtrack:
                rho parameter for the backtracking algorithm
            c_backtrack:
                c parameter for the backtracking algorithm
            mod_Newton_beta
                beta parameter for Algorithm 3.3 of modified Newtons
    '''
    # we first need to create an empty dictionary before we can add entries
    options = {}

    # Now set a default value for each of the options
    options['max_iter'] = 1e6
    options['tol'] = 1e-6
    options['step_type'] = 'Newton'
    options['init_alpha'] = 1.
    options['suff_decrease_c1'] = 1e-4
    options['output_level'] = 2
    options['rho_backtrack'] = 0.5
    options['c_backtrack'] = 1e-4
    options['mod_Newton_beta'] = 1e-4

    return options

# Rosenbrock objective function

In [6]:
class Rosenbrock:
    '''
    Implementation of the Rosenbrock objective function

    a*(x[1] - x[0]**2)**2 + (b - x[0])**2

    Parameters:
        a: scalar, see formula
        b: scalar, see formula
    '''

    def __init__(self, a=100.0, b=1.0):
        '''
        Initialize object with parameters a and b in the formula.
        '''
        self._a = a
        self._b = b

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        a = self._a
        b = self._b
        val = a*(x[1] - x[0]**2)**2 + (b - x[0])**2
        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        a = self._a
        b = self._b
        '''
        Compute the gradient in terms of paramters a and b
        '''
        grad = np.array([(-4*a*x[0]*(x[1] - x[0]**2)) - (2*(b - x[0])), 2*a*(x[1]-x[0]**2)])
        return grad

    def hessian(self, x):
        '''
        Compute the Hessian of the objective function at point x
        '''
        a = self._a
        Hess = np.array([[-4*a*x[1] + 12*a*(x[0]**2) + 2, -4*a*x[0]], [-4*a*x[0], 2*a]])
        return Hess

# 1st exponential-based objective function

In [7]:
class ExpoFunc1:
    '''
    Implementation of the 1st exponential-based objective function
    '''

    def __init__(self, a=1., b=1.):
        '''
        
        '''
        self._a = a
        self._b = b

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        val = ((x[0]-1)**2)+((np.exp(x[1])-1)*((np.exp(x[1])+1)**(-1))) + 0.1*np.exp(-1*x[1])
        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        grad = np.array([2*(x[0]-1), -0.1*np.exp(-1*x[1]) + 2*np.exp(x[1])*((np.exp(x[1])+1)**(-2))])
        return grad

    def hessian(self, x):
        '''
        Compute Hessian of objective function at point x
        '''
        Hess = np.array([[2, 0], [0, 0.1*np.exp(-1*x[1]) - (2*np.exp(x[1])*(np.exp(x[1])-1))*((np.exp(x[1])+1)**(-3))]])
        return Hess

# 2nd exponential-based objective function

In [8]:
class ExpoFunc2:
    '''
    Implementation of the 2nd exponential-based objective function
    '''

    def __init__(self, a=1., b=1.):
        '''
        
        '''
        self._a = a
        self._b = b

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        val = ((x[0]-1)**4)+((np.exp(x[1])-1)*((np.exp(x[1])+1)**(-1))) + 0.1*np.exp(-1*x[1])
        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        grad = np.array([4*((x[0]-1)**3), -0.1*np.exp(-1*x[1]) + 2*np.exp(x[1])*((np.exp(x[1])+1)**(-2))])
        return grad

    def hessian(self, x):
        '''
        Compute Hessian of objective function at point x
        '''
        Hess = np.array([[12*((x[0]-1)**2), 0], [0, 0.1*np.exp(-1*x[1]) - (2*np.exp(x[1])*(np.exp(x[1])-1))*((np.exp(x[1])+1)**(-3))]])
        return Hess

# Extended Rosenbrock objective function

In [9]:
class ExtRosenbrock:
    '''
    Implementation of the extended Rosenbrock objective function


    Parameters:
        a: scalar, see formula
        b: scalar, see formula
        n: integer, see formula
    '''

    def __init__(self, a=100.0, b=1.0, n=10):
        '''
        Initialize object with parameters a, b and n in the formula.
        '''
        self._a = a
        self._b = b
        self._n = n

    def value(self, x):
        '''
        Compute value of objective function at point x
        '''
        a = self._a
        b = self._b
        n = self._n
        
        #Initialize val at 0
        val = 0
        #loop through and sum the components of the function
        for i in range(0,n-1):
            tempval = 100*(x[i+1] - x[i]**2)**2 + (b - x[i])**2
            val += tempval
        return val

    def gradient(self, x):
        '''
        Compute gradient of objective function at point x
        '''
        a = self._a
        b = self._b
        n = self._n
        '''
        Compute the gradient in terms of paramters a, b and n
        '''
        
        #Initialize with the first value in the gradient vector
        grad = np.array([(-4*a*x[0]*(x[1] - x[0]**2)) - (2*(b - x[0]))])
        #Append successive values to the end of the gradient vector
        for i in range(1,n-1):
            tempval = np.array((-4*a*x[i]*(x[i+1] - x[i]**2)) - (2*(b - x[i])))
            grad = np.append(grad, tempval)
        #Append the nth value
        tempval = np.array(2*a*(x[n-1] - x[n-2]**2))
        grad = np.append(grad, tempval)
        return grad

    def hessian(self, x):
        '''
        Compute Hessian of objective function at point x
        '''
        a = self._a
        b = self._b
        n = self._n
        
        #Initialize an nxn Hessian of 0s
        Hess = np.array([[0]*n]*n)
        #Add values where necessary, starting with diagonals
        Hess[0,0] = -4*a*x[1] + 12*a*(x[0]**2) + 2
        for i in range(1,n-1):
            Hess[i,i] = -4*a*x[i+1] + 12*a*(x[i]**2) + 2 + 2*a
        Hess[n-1,n-1] = 2*a    
        #Add values for i,j entries of Hessian, which follow the same formula
        for i in range(0,n-1):
            for j in range(0,n):
                if i == j-1:
                    Hess[i,j] = -4*a*x[i]
                    Hess[j,i] = -4*a*x[i]
        
        return Hess

### Newton's algorithm, with gradient descent and modified Newton's algorithm allowed

In [10]:
def minimizeObjective(x_start, objFunc, options):
    '''
    Optimization method for unconstrained optimization

    Input arguments:
        x_start:
            Starting point
        objFunc:
            Objective function object.  It must have the following methods,
            where x is the point where the quantity should be evaluated.

            val = value(x)
                returns value of objective function at x
            grad = gradient(x)
                returns gradient of objective function at x
            Hess = hessian(x)
                return Hessian of objective function at x
        options:
            This is a dictionary with options for the algorithm.
            For details see the minimizeObjInitOptions function.

    Return values:
        status:
           Return code indicating reason for termination:
            0:  Optimal solution found (convergence tolerance satisfied)
           -1:  Maximum number of iterations exceeded
           -2:  Search direction is not a descent direction
          -99:  Unknown error (bug?)
        x_sol:
            Approximate critical point (or last iterate if there is a failure)
        f_sol:
            objective function value at x_sol
        stats:
            Dictionary with statistics for the run.  Its fields are
            norm_grad      Norm of gradient at final iterate
            num_iter       Number of iterations taken
            num_func_evals Number of function evaluations needed
    '''

    # initialize iteration counter
    iter_count = 0

    # initialize return flag (set to -99 so that we do not accidentally
    # declare success.)
    status = -99
    
    # get parameters out of the options dictionary
    max_iter = options['max_iter']
    tol = options['tol']
    step_type = options['step_type']
    init_alpha = options['init_alpha']
    output_level = options['output_level']
    rho = options['rho_backtrack']
    c_btrack = options['c_backtrack']
    beta = options['mod_Newton_beta']

    # initialize current iterate
    #
    # ****Important note:****
    # we need to make a copy of the x_start array.  If we would
    # just write "x_k = x_start", then the Python variables x_k and x_start
    # would be different names for the same array, and then changing
    # x_k would also change x_start.  Since we do not want to change what is
    # stored as the starting point, we need to make a copy here.
    x_k = np.copy(x_start)

    # initialize current function value, gradient, and infinity norm of the
    # gradient
    f_k = objFunc.value(x_k)
    grad_k = objFunc.gradient(x_k)
    norm_grad_k = np.linalg.norm(grad_k, np.inf)

    # initialize counter for function evaluations
    num_func_evals = 1

    # determine how many variables are in the problem
    num_vars = len(x_start)

    # initialize search direction and its length to zero
    p_k = np.zeros(num_vars)
    norm_pk = 0.0

    # initialize step size to zero (this is only necessary so that we can print
    # an output line before a meaningful step size has actually been computed.)
    alpha_k = 0.0

    # initialize the counter for the function evaluations needed in a
    # particular iteration
    num_func_it = 0

    # Print header and zero-th iteration for output
    if output_level >= 2:
        # (This is just a fancy way to create a string.  The '%s' formatting
        # makes it easy to align the header with the actual output.)
        output_header = '%6s %23s %9s %9s %6s %9s' % \
            ('iter', 'f', '||p_k||', 'alpha', '#func', '||grad_f||')
        print(output_header)
        print('%6i %23.16e %9.2e %9.2e %6i %9.2e' %
              (iter_count, f_k, norm_pk, alpha_k, num_func_it, norm_grad_k))

    ###########################
    # Beginning of main Loop
    ###########################

    # We continue the main loop until the termination tolerance is met.
    while 1:

        ##############################################################
        # Check termination tests
        ##############################################################
        if norm_grad_k <= tol:
            # Termination tolerance met
            status = 0
            break

        if iter_count >= max_iter:
            # Set flag to indicate the maximum number of iterations has been
            # exceeded
            status = -1
            # The following command says that we now want to leave the current
            # loop (which is the while loop here).  The program execution will
            # resume immediately after the end of the loop
            break

        ##############################################################
        # Compute search direction
        ##############################################################
        #Netwons algorithm
        if step_type == 'Newton':
            #Retrieve the Hessian at the current x
            hess_k = objFunc.hessian(x_k)
            #We need to calculate the inverse
            #...without calculating the ACTUAL inverse - very important!
            #Perform Cholesky decomposition to obtain p_k
            L = scipy.linalg.cholesky(hess_k, lower=True)
            L_star = np.transpose(L)
            y_chol = np.linalg.solve(L,-1*grad_k)
            p_k = np.linalg.solve(L_star,y_chol)
        
        #gradient descent algorithm
        elif step_type == 'gradient_descent':
            p_k = (-1)*grad_k
        
        #modified Newtons algorithm
        elif step_type == 'mod_Newton':
            #initialize tau
            tau_k = 0
            #check the diagonals of the hessian and adjust tau accordingly
            hess_k = objFunc.hessian(x_k)
            if min(np.diagonal(hess_k)) < 0:
                tau_k = -1*min(np.diagonal(hess_k)) + beta
            #attempt to perfrom the Cholesky decomposition
            L_boolean = False
            while L_boolean == False:
                try:
                    #If the Hessian was already positive definite, tau would be zero, L will compute,
                    # and all this error-handling will be unnecessary
                    L = scipy.linalg.cholesky(hess_k + np.identity(len(hess_k))*tau_k, lower=True)
                    L_boolean = True
                except np.linalg.LinAlgError:
                    L_boolean = False
                    tau_k = max(2*tau_k,beta)
                    pass
            #Now that we've found an acceptable L, we can proceed with the Cholesky decomposition
            L_star = np.transpose(L)
            y_chol = np.linalg.solve(L,-1*grad_k)
            p_k = np.linalg.solve(L_star,y_chol)
        else:
            raise ValueError('Invalid value for options[step_type]')

        ##############################################################
        # Perform the backtracking Armijo line search
        ##############################################################

        # initialize step size
        alpha_k = init_alpha
        
        # Compute trial point and objective value at trial point
        x_trial = x_k + alpha_k*p_k
        f_trial = objFunc.value(x_trial)
        num_func_it = num_func_it + 1
        #Establish while condition and loop until Wolfe condition satisfied
        #Backtracking parameters are pulled from the options dictionary
        while f_trial > f_k + c_btrack*alpha_k*np.dot(grad_k, p_k):
            alpha_k = rho*alpha_k
            #Recompute the trial point and obj value
            x_trial = x_k + alpha_k*p_k
            f_trial = objFunc.value(x_trial)
            #Increase the iterate count
            num_func_it = num_func_it + 1

        # Update iterate
        x_k = np.copy(x_trial)
        f_k = np.copy(f_trial)

        # Compute gradient and its norm at the new iterate
        grad_k = objFunc.gradient(x_k)
        norm_grad_k = np.linalg.norm(grad_k, np.inf)

        # For the output, compute the norm of the step
        norm_pk = np.linalg.norm(p_k, np.inf)

        # Update counter for total number of function evaluations
        num_func_evals += num_func_it

        # Increase the iteration counter
        iter_count += 1

        # Iteration output
        if output_level >= 2:
            # Print the output header every 10 iterations
            if iter_count % 10 == 0:
                print(output_header)
            print('%6i %23.16e %9.2e %9.2e %6i %9.2e' %
                  (iter_count, f_k, norm_pk, alpha_k, num_func_it, norm_grad_k))

    ###########################
    # End of main loop
    ###########################

    ###########################
    # Finalize results
    ###########################

    # Set last iterate as the one that is returned, together with its objective
    # value
    x_sol = x_k
    f_sol = f_k

    # Set the statistics
    stats = {}
    stats['num_iter'] = iter_count
    stats['norm_grad'] = norm_grad_k
    stats['num_func_evals'] = num_func_evals

    # Final output message
    if output_level >= 1:
        print('')
        print('Final objective.................: %g' % f_sol)
        print('||grad|| at final point.........: %g' % norm_grad_k)
        print('Number of iterations............: %d' % iter_count)
        print('Number of function evaluations..: %d' % num_func_evals)
        print('')
        if status == 0:
            print('Exit: Critical point found.')
        elif status == -1:
            print('Exit: Maximum number of iterations (%d) exceeded.' %
                  iter_count)
        elif status == -2:
            print('Exit: Search direction is not a descent direction.')
        elif status == -3:
            print('Exit: Exit: Step size becomes zero.')
        else:
            print('ERROR: Unknown status value: %d\n' % status)

    # Return output arguments
    return status, x_sol, f_sol, stats


# Experiment 1: Rosenbrock with x_0 = (1.2,1.2)

### Steepest descent

In [4]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
x_start = np.array([1.2, 1.2])
objFunc = Rosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 6.71775e-13
#||grad|| at final point.........: 9.9721e-07
#Number of iterations............: 12960
#Number of function evaluations..: 832287653
#Exit: Critical point found. 
#x_sol: array([1.00000082, 1.00000164])

NameError: name 'minimizeObjInitOptions' is not defined

### Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
x_start = np.array([1.2, 1.2])
objFunc = Rosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 1.08829e-25
#||grad|| at final point.........: 1.28799e-11
#Number of iterations............: 8
#Number of function evaluations..: 44
#Exit: Critical point found.
#x_sol: array([1., 1.])

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
x_start = np.array([1.2, 1.2])
objFunc = Rosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 1.08829e-25
#||grad|| at final point.........: 1.28799e-11
#Number of iterations............: 8
#Number of function evaluations..: 44
#Exit: Critical point found.
#x_sol: array([1., 1.])

##### Experiment 1 Comments
We observe that all three methods convered to a critical point solution of $[1,1]$. Additionally, the Newton methods behaved the same, implying no adjustments were applied to the Hessian in any of the modified Newton iterations.

However, we notice a marked difference in speed and computational efficiency, with the the steepest descent algorithm requiring 12,960 iterations, and 832,287,653 function evaluations, to achieve what the Newton-based methods did in 8 iterations. This large gap in efficiency illustrates the advantage of utilizing information about the curvature of $f(x)$ in determining search directions.

# Experiment 2: Rosenbrock with x_0 = (-1.2,1)

### Steepest descent

In [4]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
x_start = np.array([-1.2, 1.])
objFunc = Rosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 6.92304e-13
#||grad|| at final point.........: 9.92743e-07
#Number of iterations............: 13680
#Number of function evaluations..: 929351532
#Exit: Critical point found.
#x_sol: array([0.99999917, 0.99999833])

NameError: name 'minimizeObjInitOptions' is not defined

### Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
x_start = np.array([-1.2, 1])
objFunc = Rosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 3.74398e-21
#||grad|| at final point.........: 3.7325e-10
#Number of iterations............: 21
#Number of function evaluations..: 344
#Exit: Critical point found.
#x_sol: array([1., 1.])

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
x_start = np.array([-1.2, 1.])
objFunc = Rosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 3.74398e-21
#||grad|| at final point.........: 3.7325e-10
#Number of iterations............: 21
#Number of function evaluations..: 344
#Exit: Critical point found.
#x_sol: array([1., 1.])

##### Experiment 2 Comments
Since all we are changing from Experiment 1 is the initial search solution, it will be interesting to compare the methods' performances in Experiment 2 as compared with their corresponding performances in Experiment 1.

First analyzing the performance of the steepest descent algorithm, we observe a modest increase in iterations from  $12,960$ to $13,680$ - only a 5.3% increase due to a small change in initial search solution. Looking at our Newton-based methods, however (which again mirrored each other, likely due to a lack of need for Hessian modifications), we see an uptick in iterations from $8$ to $21$, signifying a doubling in the number of iterates required to acceptably converge. The sensitivity of Newton's method to the initial search solution is readily apparent. 

# Experiment 3: 1st exponential-based function with x_0 = (-1,1)

### Steepest descent

In [4]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
x_start = np.array([-1., 1.])
objFunc = ExpoFunc1()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: -0.205573
#||grad|| at final point.........: 8.90619e-07
#Number of iterations............: 22
#Number of function evaluations..: 268
#Exit: Critical point found.
#x_sol: array([ 1., -1.2447683])

NameError: name 'minimizeObjInitOptions' is not defined

### Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
x_start = np.array([-1., 1])
objFunc = ExpoFunc1()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#LinAlgError: 2-th leading minor of the array is not positive definite
#objFunc.hessian(x_start)
    #array([[ 2.        ,  0.        ],
           #[ 0.        , -0.14492755]])

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
x_start = np.array([-1., 1.])
objFunc = ExpoFunc1()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: -0.205573
#||grad|| at final point.........: 2.06291e-07
#Number of iterations............: 5
#Number of function evaluations..: 66
#Exit: Critical point found.
#x_sol: array([ 1., -1.24477034])

##### Experiment 3 Comments
Running the steepest descent method with this new objective function that has a lot of different exponential terms, we see a relatively quick convergence in only $22$ iterations, implying that more sophisticated algorithms should have an even easier time of handling this problem. Our non-modified Newton's method, however, returns a linear algebra error, and upon further inspection we notice that the hessian of our objective function, evaluated at this initial point, is not positive definite, as determined by the negative value in the diagonal. This situation prevents the working ability of this method.

Upon allowing Hessian modifications, however, we see that the modified Newton's method was able to converge to a solution in only $5$ iterations.

# Experiment 4: 2nd exponential-based function with x_0 = (-1,1)

### Steepest descent

In [4]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
x_start = np.array([-1., 1.])
objFunc = ExpoFunc2()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: -0.205573
#||grad|| at final point.........: 6.65928e-07
#Number of iterations............: 49
#Number of function evaluations..: 4407
#Exit: Critical point found.
#x_sol: array([ 1., -1.24476872])

NameError: name 'minimizeObjInitOptions' is not defined

### Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
x_start = np.array([-1., 1])
objFunc = ExpoFunc2()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#LinAlgError: 2-th leading minor of the array is not positive definite
#objFunc.hessian(x_start)
#    array([[ 2.        ,  0.        ],
#           [ 0.        , -0.14492755]])

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
x_start = np.array([-1., 1.])
objFunc = ExpoFunc2()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: -0.205573
#||grad|| at final point.........: 3.80733e-07
#Number of iterations............: 16
#Number of function evaluations..: 297
#Exit: Critical point found.
#x_sol: array([ 0.99543417, -1.24476995])

##### Experiment 4 Comments
The only change from Experiment 3 to Experiment 4 is a single quadratic term becoming a fourth-order term. Given the effect we would expect this type of change to have on the Hessian, it makes sense to see an uptick in the number of iterations required for the modified Newton's method to converge.

What is more interesting, though, is the effect this change had on the steepest descent method - convergence required more than twice as many iterations. The descent method seemingly focused more on the steeper gradient generated by the fourth-order term, despite the fact that $x_1$ was only included in one term of the objective function.

Collectively, these changes reflect the effect even small gradient changes can have on the ability of our line search methods to find efficient search directions.

# Experiment 5: extended Rosenbrock function with n=10, x_0 = (-1,...,-1)

### Steepest descent

In [4]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
#Construct our initial x
n = 10
x_start = np.array([-1]*n)
objFunc = ExtRosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 2.0601e-12
#||grad|| at final point.........: 9.92607e-07
#Number of iterations............: 14643
#Number of function evaluations..: 1059596858
#Exit: Critical point found.
#x_sol: 
#array([0.99999999, 0.99999999, 0.99999998, 0.99999996, 0.99999992,
#       0.99999985, 0.99999969, 0.99999938, 0.99999876, 0.99999751])

NameError: name 'minimizeObjInitOptions' is not defined

### Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
options['max_iter'] = 1e2
#Construct our initial x
n = 10
x_start = np.array([-1]*n)
objFunc = ExtRosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 113.401
#||grad|| at final point.........: 15.7087
#Number of iterations............: 100
#Number of function evaluations..: 138964
#Exit: Maximum number of iterations (100) exceeded.
#x_sol: 
#array([ 0.32115112,  0.09097437, -0.0172405 ,  0.08747292,  0.21818123,
#        0.20968267,  0.0927851 ,  0.00639282, -0.97545577,  0.95113202])

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
options['max_iter'] = 1e2
#Construct our initial x
n = 10
x_start = np.array([-1]*n)
objFunc = ExtRosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 113.401
#||grad|| at final point.........: 15.7087
#Number of iterations............: 100
#Number of function evaluations..: 138964
#Exit: Maximum number of iterations (100) exceeded.
#x_sol: 
#array([ 0.32115112,  0.09097437, -0.0172405 ,  0.08747292,  0.21818123,
#        0.20968267,  0.0927851 ,  0.00639282, -0.97545577,  0.95113202])

##### Experiment 5 Comments
Switching gears to the extended Rosebrock function, we first note that the performance of the steepest descent method is relatively unaffected in the switch from $2$ to $10$ dimensions - our iterations only increase from $12,960$ to $14643$, whereas from the infamous "curse of dimensionality" we would anticipate a much larger increase in the number of iterations required to reach convergence.

But avoiding some of the hang-ups of the shape of the Rosenbrock function seems to be beneficial in this case: our Newton-based methods did not converge, and instead seemed to become trapped at a location with a positive gradient. Both methods mirrored each other, so the positive definite-ness of the Hessian does not seem to have been a problem. Given how the difference between the steepest descent method and the Newton methods rests on the Hessian, though, it would appear that either of the following may be true:

$\quad$ a) The shape of the Rosenbrock function in larger dimensions leads to an unwieldy Hessian that causes extremely small step sizes, and hence leaves us in areas with positive gradients. Looking at the individual iteration results, we do indeed see improvements in the overall objective function, but after about $30$ iterations, the improvements are on the order of $10^{-15}$, implying that the methods would perhaps indeed converge if give enough iterations (beyond the capacity of my laptop).

$\quad$ b) The calculation used for the Hessian in the class definition of ExtRosenbrock() is faulty or miscalculated (certainly a distinct possibility!).

Of important consideration, too, is the final value of the objective function - see the comment below Experiment 8 for a discussion of this element.

# Experiment 6: extended Rosenbrock function with n=10, x_0 = (2,...,2)

### Steepest descent

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
#Construct our initial x
nvar = 10
x_start = np.array([2]*nvar)
objFunc = ExtRosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 2.02967e-12
#||grad|| at final point.........: 9.95519e-07
#Number of iterations............: 13919
#Number of function evaluations..: 963405383
#Exit: Critical point found.
#x_sol: 
#array([1.        , 1.00000001, 1.00000002, 1.00000004, 1.00000008,
#       1.00000015, 1.00000031, 1.00000062, 1.00000123, 1.00000247])

### Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
options['max_iter'] = 1e2
#Construct our initial x
nvar = 10
x_start = np.array([2]*nvar)
objFunc = ExtRosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 1.68495
#||grad|| at final point.........: 21.8343
#Number of iterations............: 100
#Number of function evaluations..: 245332
#Exit: Maximum number of iterations (100) exceeded.
#x_sol: 
#array([0.99721777, 0.96336326, 0.98454016, 0.97900323, 0.94710425,
#       0.88145166, 0.76007831, 0.55273574, 0.25281819, 0.04990802])

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
options['max_iter'] = 1e2
#Construct our initial x
nvar = 10
x_start = np.array([2]*nvar)
objFunc = ExtRosenbrock()

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 1.68495
#||grad|| at final point.........: 21.8343
#Number of iterations............: 100
#Number of function evaluations..: 245332
#Exit: Maximum number of iterations (100) exceeded.
#x_sol: 
#array([0.99721777, 0.96336326, 0.98454016, 0.97900323, 0.94710425,
#       0.88145166, 0.76007831, 0.55273574, 0.25281819, 0.04990802])

##### Experiment 6 Comments
The only change from Experiment 5 is the initial solution point, and we see a modest improvement in the steepest descent method upon this change ($13,919$ iterations, a 4.9% reduction from $14,642$ iterations). We still observe with our Newton methods, however, the same inertia in improvement after about $10$ iterations, with the discussion points from Experiment 5 remaining pertinent for Experiment 6.

# Experiment 7: extended Rosenbrock function with n=100, x_0 = (-1,...,-1)

### Steepest descent

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
#Construct our initial x
nvar = 100
x_start = np.array([-1]*nvar)
objFunc = ExtRosenbrock(a=100,b=1,n=nvar)

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 2.06718e-12
#||grad|| at final point.........: 9.96223e-07
#Number of iterations............: 15336
#Number of function evaluations..: 1181346295
#Exit: Critical point found.
#x_sol: 
#array([1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 1.        , 1.        , 1.        , 1.        ,
#       1.        , 0.99999999, 0.99999998, 0.99999996, 0.99999992,
#       0.99999985, 0.99999969, 0.99999938, 0.99999876, 0.99999751])

### Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
options['max_iter'] = 40
#Construct our initial x
nvar = 100
x_start = np.array([-1]*nvar)
objFunc = ExtRosenbrock(a=100,b=1,n=nvar)

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 251.867
#||grad|| at final point.........: 10.1008
#Number of iterations............: 40
#Number of function evaluations..: 5290
#Exit: Maximum number of iterations (40) exceeded.
#x_sol: 
#array([ 0.67280312,  0.41269993,  0.11358598, -0.01547916,  0.02405063,
#        0.03167903,  0.0320081 ,  0.03206548,  0.03206992,  0.03216049,
#        0.03214279,  0.03212917,  0.03213202,  0.03213147,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#       0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213155,  0.03213152,  0.03213159,  0.03213139,  0.03213193,
#       0.03213053,  0.0321339 ,  0.03212402,  0.03210004,  0.03206758,
#       0.03208163,  0.03207614,  0.03221117,  0.03130586,  0.03292693,
#        0.02853627,  0.03572528,  0.01629274,  0.03764733, -0.02195437,
#        0.03496872, -0.05777477,  0.00341973, -1.21059804,  1.46532997])

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
options['max_iter'] = 40
#Construct our initial x
nvar = 100
x_start = np.array([-1]*nvar)
objFunc = ExtRosenbrock(a=100,b=1,n=nvar)

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 251.867
#||grad|| at final point.........: 10.1008
#Number of iterations............: 40
#Number of function evaluations..: 5290
#Exit: Maximum number of iterations (40) exceeded.
#x_sol: 
#array([ 0.67280312,  0.41269993,  0.11358598, -0.01547916,  0.02405063,
#        0.03167903,  0.0320081 ,  0.03206548,  0.03206992,  0.03216049,
#        0.03214279,  0.03212917,  0.03213202,  0.03213147,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#       0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#       0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213154,  0.03213154,  0.03213154,  0.03213154,  0.03213154,
#        0.03213155,  0.03213152,  0.03213159,  0.03213139,  0.03213193,
#        0.03213053,  0.0321339 ,  0.03212402,  0.03210004,  0.03206758,
#        0.03208163,  0.03207614,  0.03221117,  0.03130586,  0.03292693,
#        0.02853627,  0.03572528,  0.01629274,  0.03764733, -0.02195437,
#        0.03496872, -0.05777477,  0.00341973, -1.21059804,  1.46532997])

##### Experiment 7 Comments
Again, the effect on the steepest descent method of increasing the dimensionality is surprisingly lower than one might anticipate, with our iterations only increasing from $13,919$ to $15,336$, a 10.2% increase after a 10-fold increase in dimensionality.

Once again, however, the Newton methods got "stuck" after about 20 iterations, registering scant improvement afterwards. (See below comment on Experiment 8 regarding the overall objective value.)

# Experiment 8: extended Rosenbrock function with n=10000, x_0 = (2,...,2)

### Steepest descent

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'gradient_descent'
options['max_iter'] = 3e3
#Construct our initial x
nvar = 10000
x_start = np.array([2]*nvar)
objFunc = ExtRosenbrock(a=100,b=1,n=nvar)

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 92.9406
#||grad|| at final point.........: 1.95748
#Number of iterations............: 3000
#Number of function evaluations..: 211308995
#Exit: Maximum number of iterations (3000) exceeded.

### Newton's method

In [ ]:
##### options = minimizeObjInitOptions()
options['step_type'] = 'Newton'
options['max_iter'] = 50
#Construct our initial x
nvar = 10000
x_start = np.array([2]*nvar)
objFunc = ExtRosenbrock(a=100,b=1,n=nvar)

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 2.54134
#||grad|| at final point.........: 10.29
#Number of iterations............: 50
#Number of function evaluations..: 2289
#Exit: Maximum number of iterations (50) exceeded.

### Modified Newton's method

In [ ]:
options = minimizeObjInitOptions()
options['step_type'] = 'mod_Newton'
options['max_iter'] = 50
#Construct our initial x
nvar = 10000
x_start = np.array([2]*nvar)
objFunc = ExtRosenbrock(a=100,b=1,n=nvar)

status, x_sol, f_sol, stats = minimizeObjective(x_start, objFunc, options)
x_sol

###
#Final objective.................: 2.54134
#||grad|| at final point.........: 10.29
#Number of iterations............: 50
#Number of function evaluations..: 2289
#Exit: Maximum number of iterations (50) exceeded.

##### Experiment 8 Comments
Lastly, we attempt to minimize a 10,000-variable extended Rosenbrock function, for which iterations had to be capped for processing speed considerations ($3,000$ for steepest descent, $100$ for Newton-based). It is hard to say how many more iterations would be required for the steepest descent method to converge, but using an estimated 10% increase for each 10-fold increase in variables (from Experiment 7), we should expect the steepest descent method to converge after approximately $20,000$ iterations.

The Newton methods became "stuck" after only 5 iterations in this experiment. And a very intriguing result of Experiment 8 is how much lower the final objective value is for our Newton-based methods than that of Experiment 7: $2.54$ versus $251.87$, and how similar it is to that of Experiment 6: $2.54$ versus $1.68$. This fact highlights how crucial initial starting points are for the effectiveness of the Newton methods; in the case of the extended Rosenbrock function, increasing the dimensionality from $100$ to $10,000$ had a minimal effect on the ultimate performance when the initial starting point was selected more beneficially.